In [8]:
import java.text.DecimalFormat
val f = new DecimalFormat("0.0000")


import java.text.DecimalFormat

f: DecimalFormat = java.text.DecimalFormat@674dc

In [1]:
import collection.mutable.{ArrayBuffer, HashMap}
type Attr = Int
type Index = Int
type Value = Int

case class Case(var row: ArrayBuffer[(Attr,Value)], val classLabel: Value, val frq: Int) {
  //  rowの中は特徴番号の順に昇順にソートされていると仮定する。
  //  以下のコードをいれて、ソートを仮定しなくてもよい。
  //
  //  row = row.sortWith {_._1 < _._1}
  //
  /*
   rowは、0以外の値を持つ特徴の(id, value)の並び。
   windowは、特徴全体の部分集合を指定する。
   window内では、一時的なindexが、0から順に一時的に振られる。
   windowの実態は、0以外の値を持つ特徴の(index, value)の並び
   */

  var window = row.clone()

  val size = row.size

  def apply(i: Index): Value = {
    /*
     window中でインデックスiの特徴の値
     */
    val j = this.indexOf(i)
    return if(j < 0) 0 else window(j)._2
  }

  def indexOf(i: Index): Int = {
    /*
     window中でインデックスiの特徴のwindow内での位置
     */
    var l = 0
    var u = window.size - 1
    if(u < 0 || window(l)._1 > i || window(u)._1 < i) return -1
    if(window(u)._1 == i) return u
    /*
     以下では常に、window(l)._1 <= i、かつ、window(u) > iが成り立つ。
     */
    while(true) {
      if(l + 1 == u) return if(window(l)._1 == i) l else -1
      val j = (l + u)/2
      window(j)._1 match {
        case y if y <= i => l = j
        case y if y > i => u = j
      }
    }
    0 // Syntax Errorを回避するためのダミー
  }

  def value(a: Attr): Value = {
    /*
     row中で識別子iの特徴の値
     */
    val j = this.locationOf(a)
    return if(j < 0) 0 else row(j)._2
  }

  def locationOf(a: Attr): Int = {
    /*
     row中で識別子aの特徴のrow内での位置
     */
    var l = 0
    var u = row.size - 1
    if(u < 0 || row(l)._1 > a || row(u)._1 < a) return -1
    if(row(u)._1 == a) return u
    /*
     以下では常に、row(l)._1 <= a、かつ、row(u) > aが成り立つ。
     */
    while(true) {
      if(l + 1 == u) return if(row(l)._1 == a) l else -1
      val j = (l + u)/2
      row(j)._1 match {
        case y if y <= a => l = j
        case y if y > a => u = j
      }
    }
    0 // Syntax Errorを回避するためのダミー
  }


  def renumber(order: Array[Index]) {
    /*
     order(i)は、現在のwindow中のインデックスiの特徴の、新たなwindowにおけるインデックス
     orderは、0, ..., rs-1の順列
     */
    val temp = ArrayBuffer[(Index, Value)]()
    val lim = order.size
    val ws = window.size
    var i = 0
    while(i < ws && window(i)._1 < lim) {
      val p = window(i)
      temp.append((order(p._1), p._2))
      i += 1
    }
    window = temp.sortWith(_._1 < _._1)
  }

  def compare(that: Case, index: Index): Int = {
    /*
     window内でインデックスが[0, index]内の特徴に関する辞書式順序でthisとthatを比較
     this < that => -1
     this == that => 0
     this > that => +1
     */
    var i = 0
    val lim = math.min(this.window.size, that.window.size)
    while(i < lim) {
      val x = this.window(i)
      val y = that.window(i)
      if(x._1 <= index) {
        x._1 - y._1 match {
          case diff if diff < 0 => return 1
          case diff if diff > 0 => return -1
          case _ => i += 1
        }
      } else {
        return if(y._1 <= index) -1 else 0
      }
    }
    if(i == this.window.size) {
      if(i == that.window.size) return 0
      return if(that.window(i)._1 <= index) -1 else 0
    } else {
      return if(this.window(i)._1 <= index) 1 else 0
    }
  }

 def serialize: String = row.map{x =>x._1 + ">" + x._2}.mkString(":")

}



import collection.mutable.{ArrayBuffer, HashMap}

defined type Attr
defined type Index
defined type Value
defined class Case

In [2]:
val raw_data = Array(
(ArrayBuffer((1, 1), (4, 1)), 0, 1),
(ArrayBuffer((0, 1), (1, 1), (4, 1)),1, 2),
(ArrayBuffer((0, 1),             (2, 1), (3, 1),      (5, 1)), 0, 1),
(ArrayBuffer((1, 1), (2, 1),      (4, 1)),      1, 1),
(ArrayBuffer((3, 1),      (5, 1)), 1, 1),
(ArrayBuffer((0, 1),      (2, 1),           (5, 1)), 0, 1),
(ArrayBuffer(               (3, 1), (4, 1), (5, 1)), 1, 1),
(ArrayBuffer((0, 1), (1, 1),                (5, 1)), 0, 1),
(ArrayBuffer((2, 1),      (4, 1), (5, 1)), 0, 1)
)
val data = raw_data.map(tpl => Case(tpl._1, tpl._2, tpl._3))
val nSamples = data.map(_.frq).sum

raw_data: Array[(ArrayBuffer[(Int, Int)], Int, Int)] = Array(
  (ArrayBuffer((1, 1), (4, 1)), 0, 1),
  (ArrayBuffer((0, 1), (1, 1), (4, 1)), 1, 2),
  (ArrayBuffer((0, 1), (2, 1), (3, 1), (5, 1)), 0, 1),
  (ArrayBuffer((1, 1), (2, 1), (4, 1)), 1, 1),
  (ArrayBuffer((3, 1), (5, 1)), 1, 1),
  (ArrayBuffer((0, 1), (2, 1), (5, 1)), 0, 1),
  (ArrayBuffer((3, 1), (4, 1), (5, 1)), 1, 1),
  (ArrayBuffer((0, 1), (1, 1), (5, 1)), 0, 1),
  (ArrayBuffer((2, 1), (4, 1), (5, 1)), 0, 1)
)
data: Array[Case] = Array(
  Case(ArrayBuffer((1, 1), (4, 1)), 0, 1),
  Case(ArrayBuffer((0, 1), (1, 1), (4, 1)), 1, 2),
  Case(ArrayBuffer((0, 1), (2, 1), (3, 1), (5, 1)), 0, 1),
  Case(ArrayBuffer((1, 1), (2, 1), (4, 1)), 1, 1),
  Case(ArrayBuffer((3, 1), (5, 1)), 1, 1),
  Case(ArrayBuffer((0, 1), (2, 1), (5, 1)), 0, 1),
  Case(ArrayBuffer((3, 1), (4, 1), (5, 1)), 1, 1),
  Case(ArrayBuffer((0, 1), (1, 1), (5, 1)), 0, 1),
  Case(ArrayBuffer((2, 1), (4, 1), (5, 1)), 0, 1)
)
nSamples: Int = 10

In [3]:
data.foreach{c =>
    print((0 to 5).map(i => i + ">" + c.value(i)).mkString(" "))
    println(" label>" + c.classLabel + " frq>" + c.frq)
}

0>0 1>1 2>0 3>0 4>1 5>0 label>0 frq>1
0>1 1>1 2>0 3>0 4>1 5>0 label>1 frq>2
0>1 1>0 2>1 3>1 4>0 5>1 label>0 frq>1
0>0 1>1 2>1 3>0 4>1 5>0 label>1 frq>1
0>0 1>0 2>0 3>1 4>0 5>1 label>1 frq>1
0>1 1>0 2>1 3>0 4>0 5>1 label>0 frq>1
0>0 1>0 2>0 3>1 4>1 5>1 label>1 frq>1
0>1 1>1 2>0 3>0 4>0 5>1 label>0 frq>1
0>0 1>0 2>1 3>0 4>1 5>1 label>0 frq>1


In [13]:
def log2(x: Int): Double = math.log(x)/math.log(2)
def xlog2(x: Int): Double = if(x == 0) 0 else x * math.log(x)/math.log(2)

def entropy(a: Array[Int]): Double = {
    val count = HashMap[String, Int]()
    for(c <- data) {
        val sgn = a.map(c.value(_)).mkString("")
        if(count.isDefinedAt(sgn)) count(sgn) += c.frq else count(sgn) = c.frq
    }
    return count.values.map(- xlog2(_)).sum/nSamples + log2(nSamples)
}

def entropyC(a: Array[Int]): Double = {
    val count = HashMap[String, Int]()
    for(c <- data) {
        val sgn = a.map(c.value(_)).mkString("") + c.classLabel
        if(count.isDefinedAt(sgn)) count(sgn) += c.frq else count(sgn) = c.frq
    }
    return count.values.map(- xlog2(_)).sum/nSamples + log2(nSamples)
}
val HC = entropyC(Array())
val HE = entropy((0 to 5).toArray)
val HEC = entropyC((0 to 5).toArray)
val IEC = HE + HC - HEC

defined function log2
defined function xlog2
defined function entropy
defined function entropyC
HC: Double = 1.0
HE: Double = 3.1219280948873624
HEC: Double = 3.1219280948873624
IEC: Double = 1.0

In [31]:
var prefix = Array[Int]()
var range = Array(2,3,4,5,0,1)
var relevance = range.map(f => 
                             entropyC(prefix)-entropyC(prefix:+f) - entropy(prefix)+entropy(prefix:+f))
var noise = range.map(f => -entropyC(prefix)+entropyC(prefix:+f))
var ratio = (0 until range.size).map(i => relevance(i)/noise(i))
var importance = (0 until range.size).map{i =>
    (entropy(range.take(i) ++ prefix) + HC - entropyC(range.take(i) ++ prefix))/IEC
}
println("Selected = " + prefix.mkString(" "))
println("Relevance of selected = " + f.format(entropy(prefix) + HC - entropyC(prefix)))
println("Noise of selected = " + f.format(entropyC(prefix) - HC))
println("Per feature ***")
println("Range = " + range.mkString(" "))
println("relevance = " + relevance.map(f.format(_)).mkString(" : "))
println("noise = " + noise.map(f.format(_)).mkString(" : "))
println("ratio = " + ratio.map(f.format(_)).mkString(" : "))
println("importance = " + importance.map(f.format(_)).mkString(" "))

Selected = 
Relevance of selected = -0.0000
Noise of selected = 0.0000
Per feature ***
Range = 2 3 4 5 0 1
relevance = 0.1245 : 0.0349 : 0.1245 : 0.1245 : 0.0290 : 0.0290
noise = 0.8464 : 0.8464 : 0.8464 : 0.8464 : 0.9710 : 0.9710
ratio = 0.1471 : 0.0412 : 0.1471 : 0.1471 : 0.0299 : 0.0299
importance = -0.0000 0.1245 0.3245 0.5245 0.7245 1.0000


prefix: Array[Int] = Array()
range: Array[Int] = Array(2, 3, 4, 5, 0, 1)
relevance: Array[Double] = Array(
  0.12451124978365358,
  0.0348515545596777,
  0.12451124978365358,
  0.12451124978365358,
  0.029049405545331863,
  0.02904940554533164
)
noise: Array[Double] = Array(
  0.8464393446710157,
  0.8464393446710157,
  0.8464393446710157,
  0.8464393446710157,
  0.9709505944546686,
  0.9709505944546688
)
ratio: collection.immutable.IndexedSeq[Double] = Vector(
  0.14710002620689516,
  0.04117430832946855,
  0.14710002620689516,
  0.14710002620689516,
  0.02991852078904939,
  0.029918520789049153
)
importance: collection.immutable.IndexedSeq[Double] = Vector(
  -4.440892098500626E-16,
  0.12451124978365313,
  0.32451124978365264,
  0.5245112497836533,
  0.7245112497836534,
  1.0
)

In [26]:
prefix = Array[Int](0)
range = Array(4,5,3,2)
relevance = range.map(f => 
                             entropyC(prefix)-entropyC(prefix:+f) - entropy(prefix)+entropy(prefix:+f))
noise = range.map(f => -entropyC(prefix)+entropyC(prefix:+f))
ratio = (0 until range.size).map(i => relevance(i)/noise(i))
importance = (0 until range.size).map{i =>
    (entropy(range.take(i) ++ prefix) + HC - entropyC(range.take(i) ++ prefix))/IEC
}
println("Selected = " + prefix.mkString(" "))
println("Relevance of selected = " + f.format(entropy(prefix) + HC - entropyC(prefix)))
println("Noise of selected = " + f.format(entropyC(prefix) - HC))
println("Per feature ***")
println("Range = " + range.mkString(" "))
println("relevance = " + relevance.map(f.format(_)).mkString(" : "))
println("noise = " + noise.map(f.format(_)).mkString(" : "))
println("ratio = " + ratio.map(f.format(_)).mkString(" : "))
println("importance = " + importance.map(f.format(_)).mkString(" "))

Selected = 0
Relevance of selected = 0.0290
Noise of selected = 0.9710
Per feature ***
Range = 4 5 3 2
relevance = 0.5710 : 0.4955 : 0.2955 : 0.2200
noise = 0.2755 : 0.4755 : 0.5510 : 0.7510
ratio = 2.0725 : 1.0420 : 0.5363 : 0.2929
importance = 0.0290 0.6000 0.6000 0.8000


In [28]:
prefix = Array[Int](0, 2)
range = Array(5,4,3)
relevance = range.map(f => 
                             entropyC(prefix)-entropyC(prefix:+f) - entropy(prefix)+entropy(prefix:+f))
noise = range.map(f => -entropyC(prefix)+entropyC(prefix:+f))
ratio = (0 until range.size).map(i => relevance(i)/noise(i))
importance = (0 until range.size).map{i =>
    (entropy(range.take(i) ++ prefix) + HC - entropyC(range.take(i) ++ prefix))/IEC
}
println("Selected = " + prefix.mkString(" "))
println("Relevance of selected = " + f.format(entropy(prefix) + HC - entropyC(prefix)))
println("Noise of selected = " + f.format(entropyC(prefix) - HC))
println("Per feature ***")
println("Range = " + range.mkString(" "))
println("relevance = " + relevance.map(f.format(_)).mkString(" : "))
println("noise = " + noise.map(f.format(_)).mkString(" : "))
println("ratio = " + ratio.map(f.format(_)).mkString(" : "))
println("importance = " + importance.map(f.format(_)).mkString(" "))

Selected = 0 2
Relevance of selected = 0.2490
Noise of selected = 1.7219
Per feature ***
Range = 5 4 3
relevance = 0.7510 : 0.3510 : 0.2755
noise = 0.0000 : 0.2000 : 0.2000
ratio = ∞ : 1.7549 : 1.3774
importance = 0.2490 1.0000 1.0000


In [30]:
prefix = Array[Int](0,2,5)
range = Array()
relevance = range.map(f => 
                             entropyC(prefix)-entropyC(prefix:+f) - entropy(prefix)+entropy(prefix:+f))
noise = range.map(f => -entropyC(prefix)+entropyC(prefix:+f))
ratio = (0 until range.size).map(i => relevance(i)/noise(i))
importance = (0 until range.size).map{i =>
    (entropy(range.take(i) ++ prefix) + HC - entropyC(range.take(i) ++ prefix))/IEC
}
println("Selected = " + prefix.mkString(" "))
println("Relevance of selected = " + f.format(entropy(prefix) + HC - entropyC(prefix)))
println("Noise of selected = " + f.format(entropyC(prefix) - HC))
println("Per feature ***")
println("Range = " + range.mkString(" "))
println("relevance = " + relevance.map(f.format(_)).mkString(" : "))
println("noise = " + noise.map(f.format(_)).mkString(" : "))
println("ratio = " + ratio.map(f.format(_)).mkString(" : "))
println("importance = " + importance.map(f.format(_)).mkString(" "))

Selected = 0 2 5
Relevance of selected = 1.0000
Noise of selected = 1.7219
Per feature ***
Range = 
relevance = 
noise = 
ratio = 
importance = 
